**221-35-1064** B2
**Cat Vs Dog classification**

In [ ]:
!pip install -q tensorflow keras tensorflow-datasets


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


In [ ]:
# Load Dataset

(train_raw, val_raw), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)


In [ ]:
IMG_SIZE = 160
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE


In [ ]:
# Preprocessing Function

def prepare(image, label):
    # Resize to MobileNetV2 expected input
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    # Normalize using preprocess_input
    image = preprocess_input(image)
    return image, label

In [ ]:
# Load Pretrained MobileNetV2 Base

base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)

# Freeze the base model
base_model.trainable = False

In [ ]:
# Build Classifier

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(2, activation="softmax")   # 2 classes: cat vs dog
])


In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print(">>> Training initial classifier...")
history_1 = model.fit(train_ds, validation_data=val_ds, epochs=1)

>>> Training initial classifier...
582/582 ━━━━━━━━━━━━━━━━━━━━ 63s 91ms/step - accuracy: 0.9617 - loss: 0.1027 - val_accuracy: 0.9839 - val_loss: 0.0430


In [ ]:
# Fine-Tuning

base_model.trainable = True   # Unfreeze entire model
fine_tune_at = 100            # Keep first 100 frozen

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
# Lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Add EarlyStopping callback
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

print(">>> Fine-tuning classifier...")
history_2 = model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=[early_stop])

>>> Fine-tuning classifier...
Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 74s 89ms/step - accuracy: 0.9272 - loss: 0.1770 - val_accuracy: 0.9800 - val_loss: 0.0560
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 61ms/step - accuracy: 0.9730 - loss: 0.0747 - val_accuracy: 0.9826 - val_loss: 0.0498
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 38s 62ms/step - accuracy: 0.9767 - loss: 0.0568 - val_accuracy: 0.9834 - val_loss: 0.0487
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 40s 66ms/step - accuracy: 0.9853 - loss: 0.0405 - val_accuracy: 0.9819 - val_loss: 0.0495
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 67ms/step - accuracy: 0.9867 - loss: 0.0354 - val_accuracy: 0.9800 - val_loss: 0.0515


In [ ]:
# Evaluate

loss, acc = model.evaluate(val_ds)
print(f"\nFinal Model Accuracy: {acc:.4f}")


146/146 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.9847 - loss: 0.0433

Final Model Accuracy: 0.9834
